## Dataset --> [kaggle](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow import argmax

# Data preprocessing

In [2]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()

#Retrieving the images and their labels
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        image = Image.open(path + '\\'+ a)
        image = image.resize((30,30))
        image = np.array(image)
        data.append(image)
        labels.append(i)

In [3]:
data = np.array(data)
labels = np.array(labels)

data.shape, labels.shape

((39209, 30, 30, 3), (39209,))

In [4]:
#Splitting training and validation dataset
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((31367, 30, 30, 3), (7842, 30, 30, 3), (31367,), (7842,))

In [5]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

# Model training

In [6]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]),
    Conv2D(filters=32, kernel_size=(5,5), activation='relu'),

    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),

    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),

    Flatten(),

    Dense(256, activation='relu'),
    Dropout(rate=0.5),

    Dense(43, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_val, y_val))

In [ ]:
model.save("traffic_classifier.h5")

In [7]:
model = load_model('traffic_classifier.h5')

In [ ]:
# # visualizing loss functions
# model_losses = pd.DataFrame(model.history)
# model_losses.plot()

# Model Testing

In [8]:
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values

In [9]:
test=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    test.append(np.array(image))

X_test=np.array(test)

In [10]:
pred = model.predict(X_test)

395/395 [==============================] - 16s 37ms/step


In [11]:
predicted_classes = argmax(pred, axis=1)

In [12]:
accuracy_score(predicted_classes, labels)

0.966904196357878

In [19]:
confusion_matrix(predicted_classes, labels)

array([[ 60,   0,   0, ...,   0,   0,   0],
       [  0, 713,   1, ...,   0,   0,   0],
       [  0,   5, 740, ...,   0,   0,   0],
       ...,
       [  0,   1,   0, ...,  84,   0,   0],
       [  0,   0,   0, ...,   0,  56,  26],
       [  0,   0,   0, ...,   0,   0,  61]], dtype=int64)

In [20]:
print(classification_report(predicted_classes, labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       0.99      0.94      0.97       755
           2       0.99      0.98      0.98       755
           3       0.99      0.93      0.96       479
           4       0.97      1.00      0.99       643
           5       0.95      0.98      0.96       606
           6       0.89      0.98      0.93       137
           7       0.96      1.00      0.98       434
           8       0.97      0.95      0.96       459
           9       1.00      0.98      0.99       489
          10       0.98      1.00      0.99       653
          11       0.96      0.94      0.95       428
          12       0.96      1.00      0.98       664
          13       0.99      0.99      0.99       723
          14       0.99      0.99      0.99       271
          15       1.00      0.89      0.94       235
          16       0.99      0.99      0.99       151
          17       0.99    

In [21]:
#dictionary to label all traffic signs class.
classes = { 1:'Speed limit (20km/h)',
            2:'Speed limit (30km/h)',      
            3:'Speed limit (50km/h)',       
            4:'Speed limit (60km/h)',      
            5:'Speed limit (70km/h)',    
            6:'Speed limit (80km/h)',      
            7:'End of speed limit (80km/h)',     
            8:'Speed limit (100km/h)',    
            9:'Speed limit (120km/h)',     
           10:'No passing',   
           11:'No passing veh over 3.5 tons',     
           12:'Right-of-way at intersection',     
           13:'Priority road',    
           14:'Yield',     
           15:'Stop',       
           16:'No vehicles',       
           17:'Veh > 3.5 tons prohibited',       
           18:'No entry',       
           19:'General caution',     
           20:'Dangerous curve left',      
           21:'Dangerous curve right',   
           22:'Double curve',      
           23:'Bumpy road',     
           24:'Slippery road',       
           25:'Road narrows on the right',  
           26:'Road work',    
           27:'Traffic signals',      
           28:'Pedestrians',     
           29:'Children crossing',     
           30:'Bicycles crossing',       
           31:'Beware of ice/snow',
           32:'Wild animals crossing',      
           33:'End speed + passing limits',      
           34:'Turn right ahead',     
           35:'Turn left ahead',       
           36:'Ahead only',      
           37:'Go straight or right',      
           38:'Go straight or left',      
           39:'Keep right',     
           40:'Keep left',      
           41:'Roundabout mandatory',     
           42:'End of no passing',      
           43:'End no passing veh > 3.5 tons' }

In [22]:
for i in range(10):
    # Generate a random integer between 0 and len(X_test)-1
    random_integer = tf.cast(tf.random.uniform(shape=(), minval=0, maxval=len(X_test)), tf.int32)
    print('Actual - ' + classes[labels[random_integer]] + ', Predicted - ' + classes[int(predicted_classes[random_integer])])

Actual - Veh > 3.5 tons prohibited, Predicted - Veh > 3.5 tons prohibited
Actual - Turn left ahead, Predicted - Turn left ahead
Actual - Beware of ice/snow, Predicted - Beware of ice/snow
Actual - Dangerous curve left, Predicted - Dangerous curve left
Actual - Speed limit (100km/h), Predicted - Speed limit (100km/h)
Actual - Speed limit (30km/h), Predicted - Speed limit (30km/h)
Actual - Roundabout mandatory, Predicted - Roundabout mandatory
Actual - Pedestrians, Predicted - Pedestrians
Actual - Roundabout mandatory, Predicted - Roundabout mandatory
Actual - Go straight or left, Predicted - Go straight or left
